<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%2010/cis530_nmt_skeleton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/Drive', force_remount = True)
# When you run this block, you will need to click open a link to get some 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/Drive


In [0]:
import os
# TODO
os.chdir('Drive/My Drive/CIS-530/Homework 10/data')

In [3]:
# Download data (https://nlp.stanford.edu/projects/jesc/index.html)
# TODO
!wget [a url]
!tar -xzf [a tar.gz file]

http://[a: Unterminated IPv6 numeric address.
--2020-04-16 00:23:24--  http://url%5D/
Resolving url] (url])... failed: Name or service not known.
wget: unable to resolve host address ‘url]’
tar (child): [a: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [0]:
# Preprocess data to OpenNMT format
for fname in ['train','dev','test']:
  # TODO: create 6 files, English|Japanese data for train|dev|test, one sentence per line.
  with open('split/' + fname, encoding='utf-8') as fr, open('split/en-' + fname + '.txt', 'w', encoding='utf-8') as fwe, open('split/ja-' + fname + '.txt', 'w', encoding='utf-8') as fwj:
    for line in fr:
      split_line = line.split('\t')
      english = split_line[0]
      japanese = split_line[1]
      fwe.write(english + '\n')
      fwj.write(japanese)

In [4]:
# Install BPE (https://github.com/rsennrich/subword-nmt)
# TODO
!pip install subword-nmt

In [0]:
# Apply BPE, following instructions from the Github repo
# this takes around 15 minutes

# TODO
# codes_file is where the BPE information is stored. Name it as you like. 

# Learn and apply BPE for English data

!subword-nmt learn-bpe -s 10000 < 'split/en-train.txt' > 'bpe/en_bpe' # learn from train
!subword-nmt apply-bpe -c 'bpe/en_bpe' < 'split/en-train.txt' > 'bpe/en-train-bpe-out.txt' # apply to train
!subword-nmt apply-bpe -c 'bpe/en_bpe' < 'split/en-dev.txt' > 'bpe/en-dev-bpe-out.txt' # apply to dev
!subword-nmt apply-bpe -c 'bpe/en_bpe' < 'split/en-test.txt' > 'bpe/en-test-bpe-out.txt' # apply to test

In [0]:
# Learn and apply BPE for Japanese data

!subword-nmt learn-bpe -s 10000 < 'split/ja-train.txt' > 'bpe/ja_bpe' # learn from train
!subword-nmt apply-bpe -c 'bpe/ja_bpe' < 'split/ja-train.txt' > 'bpe/ja-train-bpe-out.txt' # apply to train
!subword-nmt apply-bpe -c 'bpe/ja_bpe' < 'split/ja-dev.txt' > 'bpe/ja-dev-bpe-out.txt' # apply to dev
!subword-nmt apply-bpe -c 'bpe/ja_bpe' < 'split/ja-test.txt' > 'bpe/ja-test-bpe-out.txt' # apply to test

In [5]:
# Install OpenNMT (https://github.com/OpenNMT/OpenNMT-py)
# TODO
!pip install OpenNMT-py

     |████████████████████████████████| 194kB 22.6MB/s 
     |████████████████████████████████| 153kB 27.7MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 2.2MB 51.5MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.2.1-cp36-none-any.whl size=19266 sha256=1a843b1cb78560afc8e3b0bf0ba60e9bff090d79546f836ff818a3ac458bf16d
  Stored in directory: /root/.cache/pip/wheels/76/32/15/c284be210ac2e0b87d486c59eda286976b76275cd86f81b59d
Successfully built configargparse
ERROR: spacy 2.2.4 has requirement tqdm<5.0.0,>=4.38.0, but you'll have tqdm 4.30.0 which is incompatible.
  Found existing installation: tqdm 4.38.0
    Uninstalling tqdm-4.38.0:
      Successfully uninstalled tqdm-4.38.0
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [0]:
# Run OpenNMT, following instructions from the Github repo
# this takes 1-2 hours

# TODO
# Name the prefixes for data and checkpoints as you like
!onmt_preprocess -train_src 'bpe/ja-train-bpe-out.txt' -train_tgt 'bpe/en-train-bpe-out.txt' -valid_src 'bpe/ja-dev-bpe-out.txt' -valid_tgt 'bpe/en-dev-bpe-out.txt' -save_data 'bpe/demo' -overwrite

[2020-04-15 03:08:15,425 INFO] Extracting features...
[2020-04-15 03:08:17,815 INFO]  * number of source features: 0.
[2020-04-15 03:08:17,815 INFO]  * number of target features: 0.
[2020-04-15 03:08:17,816 INFO] Building `Fields` object...
[2020-04-15 03:08:17,816 INFO] Building & saving training data...
[2020-04-15 03:08:21,429 INFO] Building shard 0.
[2020-04-15 03:08:53,402 INFO]  * saving 0th train data shard to bpe/demo.train.0.pt.
[2020-04-15 03:09:15,602 INFO] Building shard 1.
[2020-04-15 03:09:48,290 INFO]  * saving 1th train data shard to bpe/demo.train.1.pt.
[2020-04-15 03:10:09,850 INFO] Building shard 2.
[2020-04-15 03:10:35,515 INFO]  * saving 2th train data shard to bpe/demo.train.2.pt.
[2020-04-15 03:10:52,817 INFO]  * tgt vocab size: 10099.
[2020-04-15 03:10:52,843 INFO]  * src vocab size: 19166.
[2020-04-15 03:10:53,083 INFO] Building & saving validation data...
[2020-04-15 03:10:54,295 INFO] Building shard 0.
[2020-04-15 03:10:54,323 INFO]  * saving 0th valid data s

In [0]:
!onmt_train -gpu_ranks 0 -data bpe/Demo/demo -save_model demo-model

[2020-04-15 03:11:13,030 INFO]  * src vocab size = 19166
[2020-04-15 03:11:13,032 INFO]  * tgt vocab size = 10099
[2020-04-15 03:11:13,032 INFO] Building model...
[2020-04-15 03:11:23,128 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(19166, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(10099, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
     

In [0]:
!onmt_translate -model demo-model_step_100000.pt -src 'bpe/ja-test-bpe-out.txt' -output 'bpe/pred_bpe.txt' -replace_unk -verbose

Streaming output truncated to the last 5000 lines.
PRED 751: nice girl.
PRED SCORE: -2.7891

SENT 752: ['ショーン@@', 'が@@', 'これを@@', 'くれた@@', 'んだ']
PRED 752: sea@@ n gave me this.
PRED SCORE: -2.3108

SENT 753: ['そっち@@', 'へ向か@@', 'う@@', '途中@@', 'よ', 'ベ@@', 'ック']
PRED 753: i'm on the way to you, be@@ ck.
PRED SCORE: -9.4749

SENT 754: ['そうだ', 'ベ@@', 'ン', '送@@', '電@@', '線@@', 'に従@@', 'え']
PRED 754: yes, b@@ en, follow the train.
PRED SCORE: -9.5886

SENT 755: ['情@@', 'け@@', 'をかけ@@', 'てくれる@@', 'ように@@', 'と@@', '猫@@', 'ちゃんと@@', 'ドン@@', 'キー@@', 'に@@', '祈@@', 'ってく@@', 'れ!']
PRED 755: please pray for the cat and don@@ ke@@ y!
PRED SCORE: -11.2525

SENT 756: ['「@@', 'だから@@', 'ね@@', '」@@', 'その子@@', 'に@@', '言って@@', 'あげる', '「@@', 'ママは@@', '心配@@', '性@@', 'だけど']
PRED 756: i will tell you that she is worried about her mother
PRED SCORE: -18.0330

SENT 757: ['車の@@', 'コ@@', 'マー@@', 'シャル@@', 'の?']
PRED 757: is that a car co@@ st?
PRED SCORE: -10.3872

SENT 758: ['勉強@@', 'し@@', '始めました']
PRED 758: i started 

In [0]:
# reverse BPE
!sed -r 's/(@@ )|(@@ ?$)//g' 'bpe/pred_bpe.txt' > 'bpe/pred.txt'

In [0]:
# Evaluate translations using BLEU
!wget https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
# TODO
!perl multi-bleu.perl 'split/en-test.txt' < 'bpe/pred.txt'

--2020-04-15 04:12:37--  https://raw.githubusercontent.com/moses-smt/mosesdecoder/master/scripts/generic/multi-bleu.perl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5234 (5.1K) [text/plain]
Saving to: ‘multi-bleu.perl.3’

multi-bleu.perl.3   100%[===================>]   5.11K  --.-KB/s    in 0.001s  

2020-04-15 04:12:37 (6.08 MB/s) - ‘multi-bleu.perl.3’ saved [5234/5234]

BLEU = 5.84, 28.9/11.3/5.1/2.0 (BP=0.771, ratio=0.794, hyp_len=10810, ref_len=13617)
It is not advisable to publish scores from multi-bleu.perl.  The scores depend on your tokenizer, which is unlikely to be reproducible from your paper or consistent across research groups.  Instead you should detokenize then use mteval-v14.pl, which has a standard tokenization.  Scores from multi-bleu.perl 

In [7]:
# Run NMT system on jp_test_sentences.txt 

# BPE for jpe_test_sentences.txt
!subword-nmt apply-bpe -c 'bpe/ja_bpe' < 'jp_test_sentences.txt' > 'jp-test-sentences-bpe-out.txt' # apply to test
!onmt_translate -model 'bpe/demo-model_step_100000.pt' -src 'jp-test-sentences-bpe-out.txt' -output 'translations_bpe.txt' -replace_unk -verbose
!sed -r 's/(@@ )|(@@ ?$)//g' 'translations_bpe.txt' > 'translations.txt'

Streaming output truncated to the last 5000 lines.
PRED 1251: the cat sleep@@ s under the bed.
PRED SCORE: -5.5905

SENT 1252: ['私の@@', '白い@@', 'シャツ@@', 'はどこ@@', 'ですか@@', '？']
PRED 1252: where's my white shir@@ t?
PRED SCORE: -2.3058

SENT 1253: ['彼らは@@', 'リンゴ@@', 'を取@@', 'った。']
PRED 1253: they took an ap@@ ple.
PRED SCORE: -3.0104

SENT 1254: ['私は@@', '土地@@', '線@@', 'を持って@@', 'います。']
PRED 1254: i have a land line.
PRED SCORE: -3.3947

SENT 1255: ['彼女は@@', '彼の@@', '家族@@', 'と一緒に@@', '住んで@@', 'います。']
PRED 1255: she lives with his family.
PRED SCORE: -1.3793

SENT 1256: ['彼は@@', '3人の@@', '兄弟@@', 'を持って@@', 'います。']
PRED 1256: he has three brother@@ s.
PRED SCORE: -2.6686

SENT 1257: ['歴史@@', '的な@@', '証拠@@', 'は@@', 'ありません。']
PRED 1257: there's no history of history.
PRED SCORE: -4.0018

SENT 1258: ['男は@@', '軍事@@', 'コート@@', 'を持って@@', 'います。']
PRED 1258: men have military co@@ at.
PRED SCORE: -4.6954

SENT 1259: ['それは@@', '木@@', '製@@', 'の@@', 'チェ@@', 'ーン@@', 'です。']
PRED 1259: it is a tree chain